In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.metrics import ndcg_score, dcg_score

In [ ]:
def precision_recall_fmeasure(predictions, threshold):
    predicted_and_true_ratings = defaultdict(list)

    for userId, _, true_rating, predicted_rating, _ in predictions:
        predicted_and_true_ratings[userId].append((predicted_rating, true_rating))

    precisions = []
    recalls = []
    
    for userId, user_ratings in predicted_and_true_ratings.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        num_relevant_items = sum((true_rating >= threshold) for (_, true_rating) in user_ratings)

        num_recommended_items = sum((predicted_rating >= threshold) for (predicted_rating, _) in user_ratings[:10])

        num_relevant_and_recommended_items = sum(((true_rating >= threshold) and (predicted_rating >= threshold))
                              for (predicted_rating, true_rating) in user_ratings[:10])

        current_user_precision = num_relevant_and_recommended_items / num_recommended_items if num_recommended_items != 0 else 0
        current_user_recall = num_relevant_and_recommended_items / num_relevant_items if num_relevant_items != 0 else 0

        precisions.append(current_user_precision)
        recalls.append(current_user_recall)

    precision = sum(precisions) / len(precisions)
    recall = sum(recalls) / len(recalls)
    f_measure = 2*precision*recall/(precision + recall)

    return precision, recall, f_measure  

In [ ]:
def ndcg(predictions):
    ndcgs = []

    predicted_and_true_ratings = defaultdict(list)
    for userId, _, true_rating, predicted_rating, _ in predictions:
        predicted_and_true_ratings[userId].append((predicted_rating, true_rating))
    
    for userId, user_ratings in predicted_and_true_ratings.items():
        top_items = min(len(user_ratings), 10)
    
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        model_relevance = np.asarray([[round(x[0]) for x in user_ratings][:top_items]])
        
        user_ratings.sort(key=lambda x: x[1], reverse = True)
        true_relevance = np.asarray([[round(x[1]) for x in user_ratings][:top_items]])
         
        try: 
            user_ndcg = ndcg_score(true_relevance, model_relevance)
            ndcgs.append(user_ndcg)
        except:
            pass
        
    return sum(ndcgs)/len(ndcgs)